# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 18 2023 1:01PM,261229,15,30015721,"Alliance Pharma, Inc.",Released
1,Apr 18 2023 1:01PM,261229,15,30015722,"Alliance Pharma, Inc.",Released
2,Apr 18 2023 1:01PM,261229,15,30015725,"Alliance Pharma, Inc.",Released
3,Apr 18 2023 1:01PM,261229,15,30015726,"Alliance Pharma, Inc.",Released
4,Apr 18 2023 1:01PM,261229,15,30015724,"Alliance Pharma, Inc.",Released
5,Apr 18 2023 1:01PM,261229,15,30015727,"Alliance Pharma, Inc.",Released
6,Apr 18 2023 1:01PM,261229,15,30015728,"Alliance Pharma, Inc.",Released
7,Apr 18 2023 1:01PM,261229,15,30015729,"Alliance Pharma, Inc.",Released
8,Apr 18 2023 1:01PM,261229,15,30015730,"Alliance Pharma, Inc.",Released
9,Apr 18 2023 1:01PM,261229,15,30015731,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,261334,Released,27
40,261335,Released,1
41,261336,Released,1
42,261337,Executing,1
43,261338,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261334,NaN,NaN,27.0
261335,NaN,NaN,1.0
261336,NaN,NaN,1.0
261337,NaN,1.0,NaN
261338,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,16.0,26.0,11.0
261225,3.0,24.0,3.0
261229,0.0,0.0,34.0
261231,9.0,2.0,1.0
261234,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,16,26,11
261225,3,24,3
261229,0,0,34
261231,9,2,1
261234,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,16,26,11
1,261225,3,24,3
2,261229,0,0,34
3,261231,9,2,1
4,261234,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,16,26,11
1,261225,3,24,3
2,261229,,,34
3,261231,9,2,1
4,261234,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 18 2023 1:01PM,261229,15,"Alliance Pharma, Inc."
34,Apr 18 2023 12:38PM,261338,10,Hush Hush
38,Apr 18 2023 12:26PM,261337,19,Eli Lilly and Company
39,Apr 18 2023 12:22PM,261336,10,Emerginnova
40,Apr 18 2023 12:16PM,261335,10,Alliance Pharma LTD
41,Apr 18 2023 12:13PM,261334,19,"GCH Granules USA, Inc."
68,Apr 18 2023 12:11PM,261333,10,Eye Pharma Inc
69,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line"
70,Apr 18 2023 11:57AM,261329,19,"GUSA Granules USA, Inc."
71,Apr 18 2023 11:56AM,261330,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 18 2023 1:01PM,261229,15,"Alliance Pharma, Inc.",,,34
1,Apr 18 2023 12:38PM,261338,10,Hush Hush,,,4
2,Apr 18 2023 12:26PM,261337,19,Eli Lilly and Company,,1,
3,Apr 18 2023 12:22PM,261336,10,Emerginnova,,,1
4,Apr 18 2023 12:16PM,261335,10,Alliance Pharma LTD,,,1
5,Apr 18 2023 12:13PM,261334,19,"GCH Granules USA, Inc.",,,27
6,Apr 18 2023 12:11PM,261333,10,Eye Pharma Inc,,,1
7,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line",,,1
8,Apr 18 2023 11:57AM,261329,19,"GUSA Granules USA, Inc.",,,1
9,Apr 18 2023 11:56AM,261330,10,"Senseonics, Incorporated",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 1:01PM,261229,15,"Alliance Pharma, Inc.",34,,
1,Apr 18 2023 12:38PM,261338,10,Hush Hush,4,,
2,Apr 18 2023 12:26PM,261337,19,Eli Lilly and Company,,1,
3,Apr 18 2023 12:22PM,261336,10,Emerginnova,1,,
4,Apr 18 2023 12:16PM,261335,10,Alliance Pharma LTD,1,,
5,Apr 18 2023 12:13PM,261334,19,"GCH Granules USA, Inc.",27,,
6,Apr 18 2023 12:11PM,261333,10,Eye Pharma Inc,1,,
7,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line",1,,
8,Apr 18 2023 11:57AM,261329,19,"GUSA Granules USA, Inc.",1,,
9,Apr 18 2023 11:56AM,261330,10,"Senseonics, Incorporated",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 1:01PM,261229,15,"Alliance Pharma, Inc.",34,,
1,Apr 18 2023 12:38PM,261338,10,Hush Hush,4,,
2,Apr 18 2023 12:26PM,261337,19,Eli Lilly and Company,,1,
3,Apr 18 2023 12:22PM,261336,10,Emerginnova,1,,
4,Apr 18 2023 12:16PM,261335,10,Alliance Pharma LTD,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 1:01PM,261229,15,"Alliance Pharma, Inc.",34.0,NaN,NaN
1,Apr 18 2023 12:38PM,261338,10,Hush Hush,4.0,NaN,NaN
2,Apr 18 2023 12:26PM,261337,19,Eli Lilly and Company,NaN,1.0,NaN
3,Apr 18 2023 12:22PM,261336,10,Emerginnova,1.0,NaN,NaN
4,Apr 18 2023 12:16PM,261335,10,Alliance Pharma LTD,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 1:01PM,261229,15,"Alliance Pharma, Inc.",34.0,0.0,0.0
1,Apr 18 2023 12:38PM,261338,10,Hush Hush,4.0,0.0,0.0
2,Apr 18 2023 12:26PM,261337,19,Eli Lilly and Company,0.0,1.0,0.0
3,Apr 18 2023 12:22PM,261336,10,Emerginnova,1.0,0.0,0.0
4,Apr 18 2023 12:16PM,261335,10,Alliance Pharma LTD,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3919733,49.0,13.0,0.0
15,522454,37.0,24.0,3.0
19,2612786,41.0,23.0,13.0
20,1044735,16.0,26.0,16.0
21,522639,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3919733,49.0,13.0,0.0
1,15,522454,37.0,24.0,3.0
2,19,2612786,41.0,23.0,13.0
3,20,1044735,16.0,26.0,16.0
4,21,522639,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,49.0,13.0,0.0
1,15,37.0,24.0,3.0
2,19,41.0,23.0,13.0
3,20,16.0,26.0,16.0
4,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,49.0
1,15,Released,37.0
2,19,Released,41.0
3,20,Released,16.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,0.0,3.0,13.0,16.0,0.0
Executing,13.0,24.0,23.0,26.0,1.0
Released,49.0,37.0,41.0,16.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,0.0,3.0,13.0,16.0,0.0
1,Executing,13.0,24.0,23.0,26.0,1.0
2,Released,49.0,37.0,41.0,16.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,0.0,3.0,13.0,16.0,0.0
1,Executing,13.0,24.0,23.0,26.0,1.0
2,Released,49.0,37.0,41.0,16.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()